In [25]:
import numpy as np
from IPython.display import clear_output
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.datasets import imdb
from keras.utils import np_utils
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import import_notebook
from readWrite import ReadWrite
from vectorToDoc import VectorToDoc

In [69]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"

readWrite = ReadWrite()
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

x_train = xdata[:int(len(xdata) * 0.8) + 1]
y_train = ydata[:int(len(ydata) * 0.8) + 1]
x_test = xdata[int(len(xdata) * 0.8):-1]
y_test = ydata[int(len(ydata) * 0.8):-1]

print("x_train: {}, shape: {}".format(len(x_train), x_train.shape))
print("y_train: {}, shape: {}".format(len(y_train), y_train.shape))
print("x_test: {}, shape: {}".format(len(x_test), x_test.shape))
print("y_test: {}, shape: {}".format(len(y_test), y_test.shape))

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ydata)
encoded = encoder.transform(ydata)
# convert integers to dummy variables (i.e. one hot encoded)
encoding = np_utils.to_categorical(encoded)

encoding_train = encoding[:int(len(xdata) * 0.8) + 1]
encoding_test = encoding[int(len(ydata) * 0.8):-1]
print("encoding_train shape: {}".format(encoding_train.shape))
print("encoding_test shape: {}".format(encoding_test.shape))

Read complete, total records: 1814
Read complete, total records: 1814
x_train: 1452, shape: (1452, 20)
y_train: 1452, shape: (1452,)
x_test: 362, shape: (362, 20)
y_test: 362, shape: (362,)
encoding_train shape: (1452, 3)
encoding_test shape: (362, 3)


In [86]:
vocabSize = 2000
outputDim = 64
recordsSize = 20

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

model = Sequential()
model.add(Embedding(vocabSize, outputDim, input_length=recordsSize))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, input_dim=(20,), activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 20, 64)            128000    
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               66000     
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 303       
Total params: 194,303
Trainable params: 194,303
Non-trainable params: 0
_________________________________________________________________


In [88]:
batch_size = outputDim
epochs = 5
scores = 0

while(True):        
    model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim, verbose=2)
    scores = model.evaluate(x_test, encoding_test, verbose=0)    
    print("Accuracy: %.2f%%" % (scores[1]*100))
    if(scores[1] > 0.83):
        break
    else:
        clear_output()

# model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim, verbose=2)
# scores = model.evaluate(x_test, encoding_test, verbose=0)    
# print(scores[1])


Train on 1452 samples, validate on 362 samples
Epoch 1/5
0s - loss: 6.4804e-04 - acc: 1.0000 - val_loss: 1.6894 - val_acc: 0.8039
Epoch 2/5
0s - loss: 5.9473e-04 - acc: 0.9998 - val_loss: 1.7592 - val_acc: 0.8094
Epoch 3/5
0s - loss: 5.3864e-04 - acc: 1.0000 - val_loss: 1.8133 - val_acc: 0.8103
Epoch 4/5
0s - loss: 0.0223 - acc: 0.9943 - val_loss: 1.4729 - val_acc: 0.8361
Epoch 5/5
0s - loss: 0.0287 - acc: 0.9924 - val_loss: 1.0570 - val_acc: 0.8343
Accuracy: 83.43%


In [89]:
'''
Saving model or load model
'''
# model.save('model/tweetClassifier.h5')
# model = load_model('model/tweetClassifier.h5')

In [90]:
'''
0-unknown, 1-update, 2-delay
'''
vectorsArr2 = readWrite.readFile('docVectors3.csv')
oriRecords2 = readWrite.readOriFile('user_tweets3.txt')

print("Shape: {}".format(vectorsArr2.shape))

classResult = []
predicted = model.predict_classes(vectorsArr2)
predicted = np.reshape(predicted, (predicted.size,))

print()
print(predicted.shape)

for idx, score in enumerate(predicted):    
    if(score == 2): 
        classResult.append(oriRecords2[idx])       
        print(predicted[idx])
        print(oriRecords2[idx])        

readWrite.writeFile('docVectors3Result.txt', predicted)

Read complete, total records: 689
Shape: (689, 20)
608/689 [=========================>....] - ETA: 0s
(689,)
2
@SMRT_Singapore This is very very misleading. Train services from amk to Newton are NOT available. SMRT should be c https://t.co/yf1idrHCmk

2
@SMRT_Singapore The word "update" suggests there is a change in status.

2
@SMRT_Singapore The new "revised" tweet still means no service between AMK and Newton, it a way to lessen the negativity on them, pui

2
@SMRT_Singapore can talk more rubbish??????? just fix the damn problem and stop making shit like this happen 24 fucking 7

2
@SMRT_Singapore Just reached Newton. Announcement says no service from AMK to Marina South Pier. please update your Twitter.

2
@SMRT_Singapore Separately, a fire was spotted in a tunnel between Marina Bay and Raffles Place stations. https://t.co/WKzfdv8C31

2
@SMRT_Singapore Flooded tunnels cause disruption on North-South Line, affecting service from Ang Mo Kio to Marina S https://t.co/DxT600Njz8

2
@SMRT